Assignment 5 Part A:

Aim:- Do an end2end well annotated and commented demo of continual learning using avalanche library in colab

Applying SimpleMLP modeling on PermutedMNIST(dataset)

In [ ]:
# Install Avalanche and its dependencies
!pip install git+https://github.com/ContinualAI/avalanche.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-1px96yh4
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-1px96yh4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.9 MB 34.4 MB/s 
     |████████████████████████████████| 532 kB 66.7 MB/s 
     |████████████████████████████████| 529 kB 64.5 MB/s 
     |████████████████████████████████| 427 kB 63.4 MB/s 
     |████████████████████████████████| 182 kB 66.5 MB/s 
     |████████████████████████████████| 168 kB 66.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 168 kB 69.1 MB/s 
     |████████████████████████████████| 166 kB 71.7 MB/s 
     |████████████████████████████████| 166 kB 70.5 MB/s 
     |█

In [ ]:
import torch
# use CrossEntropyLoss
from torch.nn import CrossEntropyLoss
# use stochastic GD optimizer
from torch.optim import SGD
# import the PermutedMNIST dataset
from avalanche.benchmarks.classic import PermutedMNIST
# import the SimpleMLP dataset
from avalanche.models import SimpleMLP
# we will use Naive training strategy
from avalanche.training.supervised import Naive

In [ ]:
# Configure the device settings
# check for CUDA GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model instantiation
model = SimpleMLP(num_classes = 10)

In [ ]:
# Benchmark instantiation
perm_mnist = PermutedMNIST(n_experiences=3)
# split stream into train and test set
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



In [ ]:
# define optimizer and loss function for training
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

In [ ]:
# Instantiate Continual learning strategy
strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, device=device)

In [ ]:
results = []
# train and test epochs
for train_task in train_stream:
    # training
    strategy.train(train_task, num_workers=4)
    # evaluate the model and store the results
    results.append(strategy.eval(test_stream))

-- >> Start of training phase << --
0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 1875/1875 [01:05<00:00, 28.64it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4867
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8638
100%|██████████| 1875/1875 [01:05<00:00, 28.81it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2612
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9244
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [00:05<00:00, 57.20it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1834
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9486
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 313/313 [00:05<00:00, 56.06it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 2.5753
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1156
-- S

In [ ]:
results

[{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9244,
  'Loss_Epoch/train_phase/train_stream/Task000': 0.26124954459667205,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9486,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.18343571719527244,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.1156,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp001': 2.575326578140259,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.1165,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp002': 2.6268580448150636,
  'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.3935666666666667,
  'Loss_Stream/eval_phase/test_stream/Task000': 1.7952067800501983},
 {'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.93385,
  'Loss_Epoch/train_phase/train_stream/Task000': 0.2270553907841444,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9314,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.22423036908060312,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp00

Reference: https://analyticsindiamag.com/avalanche-a-python-library-for-continual-learning/